In [1]:
# import all necessary packages & modules
import os
import time
import numpy as np
import pandas as pd
import json
import pyomo.environ as pyomo
from pyomo.opt import ProblemFormat
import matplotlib.pyplot as plt
import networkx as nx
import highspy
from networkx.readwrite import json_graph
from pyomo.opt import SolverStatus, TerminationCondition
from good_model_working import utils


import good_model_working
from good_model_working.reload import deep_reload
from good_model_working import opt_model
from good_model_working import diagnostics

In [2]:
deep_reload(good_model_working)

# Removing generators for testing
gen_to_remove = [
        'Fossil Waste', 
        #'Municipal Solid Waste', 
        'Non-Fossil Waste', 
        'Pumped Storage',
        'Fuel Cell',
        'Landfill Gas', 
        # "Energy Storage", 
        # "Solar PV", 
        # "Onshore Wind", 
        # 'New Battery Storage', 
        # 'IMPORT', 
        # 'Tires',
        'Offshore Wind', 
        'Solar Thermal'
        ]


# import the model data & sets
input_sets = utils.get_sets('/Users/peterambiel/Desktop/good_model/Model Input/all_input_sets_sorted.json')
model_sets = utils.filter_sets(input_sets, gen_to_remove)

graph = utils.create_graph('/Users/peterambiel/Desktop/good_model/Model Input/all_input_objects.json')

In [3]:
deep_reload(good_model_working)

# MODEL TESTING
'''
    To analyze specific regions, input the appropriate subregion
    to the variable sub_region below
'''

user_input = input('Please select your region(s) for analysis (separated by commas): \nAll\nFlorida\nMISO\nNew England\nPJM\nSSP\nSoutheast\nERCOT\nWECC\n')

subgraph, sub_nodes = utils.get_subgraph(user_input, graph)

constraint_to_deactivate = ['generator', 'solar', 'wind', 'storage']

model_data = {
    'test_nodes': True,
    'test_cons': False,
    'graph': graph, 
    'sets': model_sets, 
    'subgraph_nodes': sub_nodes, 
    'subgraph': subgraph,
    'gen_to_remove': gen_to_remove,
    'constraint_deactivation': None
}

In [4]:
# import good_model_working
deep_reload(good_model_working)

#Change this to point to your version of cbc or use another solver
solver_name='appsi_highs'

# run model 
problem = good_model_working.opt_model.Opt_Model(model_data, solver_name)

[+   0.29] Grid built
[+   0.00] Sets built
[+   0.02] Parameters built
[+   0.01] Variables built
[+   0.03] Objective Function built
[+   0.30] Local Constraints built
[+   0.00] Transmission constraints built
[+   0.00] Starting balanacing constraint...
[+   0.25] Balancing constraint built
[+   0.00] All Constraints built
[+   0.00] Model built


In [5]:
deep_reload(good_model_working.utils)

# Generate simple model statistics
utils.get_model_statistcs(problem.model)
utils.get_total_generator_count(problem.model)

 total_variables  total_constraints
           16392              16728
Total generators: 543.0


In [6]:
# Solve the model
problem.solve_model()

# print the objective value
print(f"Total system cost: ${pyomo.value(problem.model.obj_func):,.2f}")

Running HiGHS 1.7.0 (git hash: 27ccfaa): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e-04, 1e+00]
  Cost   [7e+00, 2e+03]
  Bound  [1e-08, 1e-08]
  RHS    [1e-01, 1e+06]
Presolving model
216 rows, 12854 cols, 17240 nonzeros  0s
144 rows, 4826 cols, 7365 nonzeros  0s
144 rows, 4826 cols, 7365 nonzeros  0s
Presolve : Reductions: rows 144(-16584); columns 4826(-11566); elements 7365(-38010)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     5.0257972755e-03 Pr: 144(711305) 0s
        144     1.6050551211e+07 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 144
Objective value     :  1.6050551212e+07
HiGHS run time      :          0.02
Total system cost: $16,050,551.21


In [7]:
deep_reload(good_model_working)

# Get model results
results = problem.get_results()

In [8]:
# get diagnostics
deep_reload(diagnostics)

hourly_mix = diagnostics.get_hourly_gen_mix(results)
annual_mix = diagnostics.get_annual_gen_mix(results)

In [9]:
# Compare 2021 US annual mix baseline to model generated fuel mix
deep_reload(diagnostics)

diagnostics.compare_annual_mix_to_baseline(annual_mix)

                        Resource Percentage_Baseline Percentage_Model
0                           Coal             21.900%          44.934%
1                            Oil              0.600%           0.000%
2                            Gas             38.400%           4.015%
3                   Other Fossil              0.500%           0.000%
4                        Nuclear             18.900%          40.666%
5                          Hydro              6.000%           9.874%
6                        Biomass              1.300%           0.511%
7                           Wind              9.200%           0.000%
8                          Solar              2.800%           0.000%
9                     Geothermal              0.400%           0.000%
10  Other unknown/purchased fuel              0.100%           0.000%


In [ ]:
# plot diagnostics
deep_reload(diagnostics)

diagnostics.plot_hourly_gen_mix(hourly_mix)